Newly introduced in **transformers v2.3.0**, pipelines provides a high-level, easy to use, API for doing inference over a variety of downstream-tasks. <br>
**Pipelines** encapsulate the overall process of every NLP process:
1. **Tokenization**: Split the initial input into multiple sub-entities with ... properties (i.e. tokens).
2. **Inference**: Maps every tokens into a more meaningful representation.
3. **Decoding**: Use the above representation to generate and/or extract the final output for the underlying task.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.3MB/s 
     |████████████████████████████████| 3.3MB 31.1MB/s 
     |████████████████████████████████| 870kB 51.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=d5af3f571f391c10dac4fc35934b721d9a085135b557852cf091cc8da27d8883
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
import transformers
from transformers import pipeline
from ipywidgets import widgets

import numpy as np

# transformers : 4.5.1  |  np : 1.19.5
print(f'transformers : {transformers.__version__}  |  np : {np.__version__}')

transformers : 4.5.1  |  np : 1.19.5


## 1. Sentence Classification - Sentiment Analysis

In [3]:
%%time
sentiment_classifier = pipeline('sentiment-analysis')


CPU times: user 6.83 s, sys: 1.08 s, total: 7.91 s
Wall time: 8.79 s


In [4]:
%%time
sent_list = [
    "That's so mean to say that way to this wonderful child",
    "This is the perfect way of ruinning a movie with such a wonderful actors",
    "I'm not sure that is really wonderful idea.",
    "How come do you think that he is a kind boy?"
]

for sent in sent_list:
  print(f'>>> {sent} :', sentiment_classifier(sent))

>>> That's so mean to say that way to this wonderful child : [{'label': 'POSITIVE', 'score': 0.9997044801712036}]
>>> This is the perfect way of ruinning a movie with such a wonderful actors : [{'label': 'NEGATIVE', 'score': 0.9989961385726929}]
>>> I'm not sure that is really wonderful idea. : [{'label': 'NEGATIVE', 'score': 0.9893096685409546}]
>>> How come do you think that he is a kind boy? : [{'label': 'NEGATIVE', 'score': 0.9777455925941467}]
CPU times: user 368 ms, sys: 5.25 ms, total: 373 ms
Wall time: 316 ms


## 2. Token Classification - Named Entity Recognition

Here is an example using the pipelines do to named entity recognition, trying to identify tokens as belonging to one of 9 classes:
- O, Outside of a named entity
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MIS, Miscellaneous entity
- B-PER, Beginning of a person’s name right after another person’s name
- I-PER, Person’s name
- B-ORG, Beginning of an organisation right after another organisation
- I-ORG, Organisation
- B-LOC, Beginning of a location right after another location
- I-LOC, Location

In [5]:
%%time
ner_model = pipeline('ner')


CPU times: user 32.3 s, sys: 4.9 s, total: 37.2 s
Wall time: 38.2 s


In [6]:
%%time
def pprint_result_list(model, sent):
  print(f'>>> {sent} :')
  for dic in model(sent):
    print('\t', dic)

sent_list = [
    'Hugging Face is a French company based in New York.',
    'My Google Workspace id is handed to Sangeun.',
    'Trump made something new for Cheoin-gu.',
    'Trump made something new for Cheoin-gu, Korea'
]

pprint_result_list(ner_model, sent_list[0])
print('\n')

ner_model.grouped_entities = True
for sent in sent_list:
  pprint_result_list(ner_model, sent)

>>> Hugging Face is a French company based in New York. :
	 {'word': 'Hu', 'score': 0.9968873858451843, 'entity': 'I-ORG', 'index': 1, 'start': 0, 'end': 2}
	 {'word': '##gging', 'score': 0.9329522848129272, 'entity': 'I-ORG', 'index': 2, 'start': 2, 'end': 7}
	 {'word': 'Face', 'score': 0.9781811237335205, 'entity': 'I-ORG', 'index': 3, 'start': 8, 'end': 12}
	 {'word': 'French', 'score': 0.9981815814971924, 'entity': 'I-MISC', 'index': 6, 'start': 18, 'end': 24}
	 {'word': 'New', 'score': 0.9987512826919556, 'entity': 'I-LOC', 'index': 10, 'start': 42, 'end': 45}
	 {'word': 'York', 'score': 0.9976728558540344, 'entity': 'I-LOC', 'index': 11, 'start': 46, 'end': 50}


>>> Hugging Face is a French company based in New York. :
	 {'entity_group': 'ORG', 'score': 0.9693402647972107, 'word': 'Hugging Face', 'start': 0, 'end': 12}
	 {'entity_group': 'MISC', 'score': 0.9981815814971924, 'word': 'French', 'start': 18, 'end': 24}
	 {'entity_group': 'LOC', 'score': 0.998212069272995, 'word': 'N

## 3. Question Answering

In [7]:
%%time
qa_model = pipeline('question-answering')


CPU times: user 7.68 s, sys: 1.04 s, total: 8.72 s
Wall time: 9.49 s


In [8]:
%%time
# Several lines from https://en.wikipedia.org/wiki/Seoul
txt = """
    Seoul (/soʊl/, like soul; Korean: 서울 [sʌ.ul] (About this soundlisten); lit. 'Capital'), officially the Seoul Special City, is the capital[7] and largest metropolis of South Korea.[8] Seoul has a population of 9.7 million people, and forms the heart of the Seoul Capital Area with the surrounding Incheon metropolis and Gyeonggi province. Seoul was the world's 4th largest metropolitan economy in 2014 after Tokyo, New York City and Los Angeles.[9] In 2017, the cost of living in Seoul was ranked the 6th highest globally.[10][11]
    With technology hubs centered in Gangnam and Digital Media City,[12] the Seoul Capital Area is home to the headquarters of 14 Fortune Global 500 companies, including Samsung,[13] LG, and Hyundai. The metropolis exerts a major influence in regional affairs as one of the five leading hosts of global conferences; as of 2018 it was ranked 3rd in the world after Singapore (1st) and Brussels (2nd).[14] Seoul has hosted the 1986 Asian Games, 1988 Summer Olympics, 2002 FIFA World Cup (with Japan), and the 2010 G-20 Seoul summit.
"""

sent_list = [
    'Answer the population of the city',
    'How many humans are there?',
    
    'Any special event that happened in Seoul?',
    'Any special event that happened in Seoul'
]

for sent in sent_list:
  print(f'>>> {sent} :')
  print('\t', qa_model(context=txt, question=sent))

>>> Answer the population of the city :
	 {'score': 0.6106517314910889, 'start': 214, 'end': 225, 'answer': '9.7 million'}
>>> How many humans are there? :
	 {'score': 0.8688034415245056, 'start': 214, 'end': 225, 'answer': '9.7 million'}
>>> Any special event that happened in Seoul? :
	 {'score': 5.868524021934718e-05, 'start': 108, 'end': 126, 'answer': 'Seoul Special City'}
>>> Any special event that happened in Seoul :
	 {'score': 2.3620407318958314e-06, 'start': 1038, 'end': 1064, 'answer': 'the 2010 G-20 Seoul summit'}
CPU times: user 2.37 s, sys: 37.7 ms, total: 2.41 s
Wall time: 1.21 s


## 4. Text Generation - Mask Filling

In [9]:
%%time
mlm_model = pipeline('fill-mask')


CPU times: user 9.36 s, sys: 1.4 s, total: 10.8 s
Wall time: 11.6 s


In [10]:
%%time
mask_token = mlm_model.tokenizer.mask_token
sent_list = [
    f'Hugging Face is a French company based in {mask_token}',
    f'Hugging Face is a {mask_token} company based in New York',
    f'I visited home to {mask_token} milk',
    f'I visited home to {mask_token} her'
]

for sent in sent_list:
  pprint_result_list(mlm_model, sent)

>>> Hugging Face is a French company based in <mask> :
	 {'sequence': 'Hugging Face is a French company based in Paris', 'score': 0.2775893807411194, 'token': 2201, 'token_str': ' Paris'}
	 {'sequence': 'Hugging Face is a French company based in Lyon', 'score': 0.14941272139549255, 'token': 12790, 'token_str': ' Lyon'}
	 {'sequence': 'Hugging Face is a French company based in Geneva', 'score': 0.045763980597257614, 'token': 11559, 'token_str': ' Geneva'}
	 {'sequence': 'Hugging Face is a French company based in France', 'score': 0.04576262831687927, 'token': 1470, 'token_str': ' France'}
	 {'sequence': 'Hugging Face is a French company based in Brussels', 'score': 0.04067569971084595, 'token': 6497, 'token_str': ' Brussels'}
>>> Hugging Face is a <mask> company based in New York :
	 {'sequence': 'Hugging Face is a cosmetics company based in New York', 'score': 0.08008444309234619, 'token': 23353, 'token_str': ' cosmetics'}
	 {'sequence': 'Hugging Face is a technology company based in N

## 5. Summarization
Summarization is currently supported by Bart and T5.

In [11]:
%%time
summarizer = pipeline('summarization')

# function to print string in pretty way
def pprint_result_string(text):
  line_nth_char = 0
  for char in text:
    print(char, end='')
    line_nth_char += 1
    
    if line_nth_char > 100 and char in [' ', '.']:
      print()
      line_nth_char = 0
    elif char == '\n':
      line_nth_char = 0
  print('\n')


CPU times: user 36.6 s, sys: 4.4 s, total: 41 s
Wall time: 41.9 s


In [12]:
%%time
# Several lines from https://www.quora.com/In-Python-vs-R-which-one-do-you-prefer-Why-What-are-the-pros-and-cons-What-can-Python-do-that-R-cannot-and-vice-versa
text = """ 
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.
"""
summarized_txt = summarizer(text)[0]['summary_text']
pprint_result_string(summarized_txt)

# 요약문 번역 결과 (구글 번역기)
# 우리는주의 메커니즘만을 기반으로하는 새로운 단순 네트워크 아키텍처 인 Transformer를 제안하며, 재발과 컨볼 루션을 완전히 제거합니다.
# 두 가지 기계 번역 작업에 대한 실험을 통해 이러한 모델은 품질이 우수하면서도 병렬화가 가능하고 학습 시간이 훨씬 단축되는 것으로 나타났습니다.
# Transformer는 영어 구성 구문 분석에 성공적으로 적용하여 다른 작업에 잘 일반화됩니다.

 We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, 
dispensing with recurrence and convolutions entirely . Experiments on two machine translation tasks show 
these models to be superior in quality while being more parallelizable and requiring significantly less 
time to train . The Transformer generalizes well to other tasks by applying it successfully to English 
constituency parsing .

CPU times: user 15.8 s, sys: 156 ms, total: 16 s
Wall time: 8 s


## 6. Translation
Translation is currently supported by T5 for the language mappings English-to-French (translation_en_to_fr), English-to-German (translation_en_to_de) and English-to-Romanian (translation_en_to_ro).

In [13]:
%%time
# English to German
translator = pipeline('translation_en_to_de')


CPU times: user 24.7 s, sys: 8.79 s, total: 33.4 s
Wall time: 35.3 s


In [14]:
%%time

src_sent = "The history of natural language processing (NLP) generally started in the 1950s, although work can be found from earlier periods."
target_sent = translator(src_sent)[0]['translation_text']
pprint_result_string(target_sent)

# 번역문 번역 결과 (구글 번역기)
# 자연어 처리 (NLP)의 역사는 일반적으로 1950 년대에 시작되었지만 초기 작업을 찾을 수 있습니다.

# 번역문 번역 결과 (네이버 파파고 번역기)
# 자연 언어 처리의 역사(NLP)는 일반적으로 1950년대에 시작되었지만, 초기 작품들은 발견될 수 있다.

Die Geschichte der natürlichen Sprachenverarbeitung (NLP) begann im Allgemeinen in den 1950er Jahren, 
obwohl Arbeit aus früheren Zeiten gefunden werden kann.

CPU times: user 6.91 s, sys: 380 ms, total: 7.29 s
Wall time: 3.84 s


## 7. Text Generation
Text generation is currently supported by GPT-2, OpenAi-GPT, TransfoXL, XLNet, CTRL and Reformer.

In [15]:
%%time
text_generator = pipeline("text-generation")


CPU times: user 15.6 s, sys: 4.15 s, total: 19.7 s
Wall time: 20.5 s


In [16]:
%%time
text_list = [text_generator("Today is a beautiful day and I will")[0]['generated_text'] for i in range(3)]

for text in text_list:
  # print('>>> ', end='')
  print('='*50, '\n')
  pprint_result_string(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Today is a beautiful day and I will make my way to the top this year."

Drayton's second game starts out just off the line of scrimmage and the Ravens have a huge need at wide 
receiver in the lineup that could open up


Today is a beautiful day and I will take all I can out of that morning."

But those inside the House have already been given a "thumbs up"

But Rep. Mark Meadows said it was not too late for the House


Today is a beautiful day and I will never stop smiling because of you for being my hero. pic.twitter.
com/ZfT5N9I2vT — Justin Bieber (@JustinBieber) November 24, 2017



CPU times: user 11.6 s, sys: 94.3 ms, total: 11.7 s
Wall time: 5.87 s
